# Generate Validation Set

## Includes

In [ ]:
# mass includes
import os, sys, warnings
import ipdb
import pickle
import torch as t
import torchvision as tv
from tqdm.notebook import tqdm

# add paths for all sub-folders
paths = [root for root, dirs, files in os.walk('.')]
for item in paths:
    sys.path.append(item)

from ipynb.fs.full.config import valConf
from ipynb.fs.full.monitor import Visualizer
from ipynb.fs.full.network import r2rNet
from ipynb.fs.full.dataLoader import fivekNight
from ipynb.fs.full.util import *

## Initialization

In [ ]:
# for debugging only
%pdb off
warnings.filterwarnings('ignore')

# choose GPU if available
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

# define model
opt = valConf()
converter = r2rNet().to(device)
converter.load('./saves')
converter.eval()

# dataloader for training
val_dataset = fivekNight(opt)
val_loader = t.utils.data.DataLoader(val_dataset, shuffle=True)

## Training entry

In [ ]:
# make new folder
save_path = os.path.join(opt.data_root, 'val%d' % opt.amp_range[1])
if not os.path.exists(save_path):
    os.makedirs(save_path)

for index, (syth_img, syth_mask) in tqdm(enumerate(val_loader), total=50):
    # copy to device
    syth_img = syth_img.to(device)
    syth_mask = syth_mask.to(device)

    # convert to training samples
    thumb_img, struct_img, seg_mask, amp, noisy_raw, sorted_mask, wb = toRaw(
        converter, syth_img, syth_mask, opt)

    # save to files
    file_path = os.path.join(save_path, 'img%04d.jpg' % (index))
    data_dict = {}
    data_dict['syth_img'] = syth_img.squeeze().cpu()
    data_dict['thumb_img'] = thumb_img.squeeze().cpu()
    data_dict['struct_img'] = struct_img.squeeze().cpu()
    data_dict['seg_mask'] = seg_mask.squeeze().cpu()
    data_dict['amp'] = amp.squeeze().cpu()
    data_dict['noisy_raw'] = noisy_raw.squeeze().cpu()
    data_dict['sorted_mask'] = sorted_mask.squeeze().cpu()
    data_dict['wb'] = wb.squeeze().cpu()
    with open(file_path.replace('jpg', 'pkl'), 'wb') as pkl_file:
        pickle.dump(data_dict, pkl_file)
    tv.utils.save_image(syth_img.squeeze(), file_path)

    if index >= 50:
        break